<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/%E6%97%A5%E7%A8%8B%E4%BB%8E%E8%81%8A%E5%A4%A9%E6%96%87%E6%9C%AC%E4%B8%AD%E6%8F%90%E5%8F%96%E4%BF%A1%E6%81%AF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:

!pip install --upgrade --quiet  pillow
!pip install langchain langchain_community
!pip install --upgrade httpx httpx-sse PyJWT
!pip install langchain langchainhub httpx_sse
!pip install --upgrade 'openai>=1.0'
!pip install langchain_openai
!pip install langchain_mistralai
# 0932017b160f1b9e35afd714225b77c0.jlo4SxWIkye84xPh

我们将使用能够调用函数/工具的聊天模型从文本中提取信息
https://python.langchain.com/v0.1/docs/use_cases/extraction/quickstart/

In [17]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()
  api_key = os.getenv("ZHIPUAI_API_KEY")


In [51]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

In [99]:
## 以上是无错的代码
from datetime import datetime
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Event(BaseModel):
    """Information about an event."""
    time: Optional[str] = Field(default=None, description="The time when the event occurred.If not, return empty")
    schedule: Optional[bool] = Field(default=None,description="如果属于日程安排,就返回True。否则返回False")
    event: Optional[bool] = Field(default=None,description="description of the event")
    robot: Optional[bool] = Field(default=None,description="ai reply")

def _get_datetime():
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")


from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field


prompt = ChatPromptTemplate.from_messages(
    [
      (
          "system",
          "你是一个日程管家。请根据文本内容提取日程相关信息。"
          "如果你无法确定是需要提取的属性值，就返回null。"
      ),
      ("human", "{text}")
    ]
)


from langchain_mistralai import ChatMistralAI

runnable = prompt | llm.with_structured_output(schema=Event)

input_text = """明天是一个未知的新开始，充满了无限可能。"""
runnable.invoke({"text": input_text})

Event(time=None, schedule=False, event=None, robot=None)

In [104]:
## 修改中的代码
from datetime import datetime
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Event(BaseModel):
    """Information about an event."""
    time: Optional[str] = Field(default=None, description="The time when the event occurred.If not, return empty")
    schedule: Optional[bool] = Field(default=None,description="如果属于日程安排,就返回True。否则返回False")

    robot: Optional[str] = Field(default=None,description="机器人回复的消息")

def _get_datetime():
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")


from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field


prompt = ChatPromptTemplate.from_messages(
    [
      (
          "system",
          "你是一个日程管家。请根据文本内容提取日程相关信息。"
          f"现在时间是：{_get_datetime()}"
          "如果你无法确定是需要提取的属性值，就返回null。"
          "将AI的回复写到robot属性上"
      ),
      ("human", "{text}")
    ]
)


from langchain_mistralai import ChatMistralAI

runnable = prompt | llm.with_structured_output(schema=Event)

input_text = """明天是一个未知的新开始，充满了无限可能。"""
runnable.invoke({"text": input_text})

Event(time='', schedule=False, robot='明天是一个未知的新开始，充满了无限可能。')

In [105]:
#日程按排的测试
input_text = """帮我查询人民日报金句，主题：青春，请至少提供5句。"""
runnable.invoke({"text": input_text})

Event(time='2024-07-02 09:23:48', schedule=None, robot='好的，请稍等，我需要调用搜索引擎来查询相关信息。')

In [108]:
#日程按排的测试 II
input_text = """你是一个非常高情商会说话的人，跟任何人说话分寸都把握的很好，让人如沐春风，下面请你根据我提供的场景与我展开对话
场景：上班时在电梯和领导相遇;要求：你是公司员工，我是老板，你来和我对话，让我对你留下很好的印象"""
runnable.invoke({"text": input_text})

In [48]:
from datetime import datetime
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field


class Event(BaseModel):
    """Information about an event."""
    time: Optional[str] = Field(default=None, description="The time when the event occurred")
    schedule: Optional[bool] = Field(default=None,description="如果属于日程安排,就返回True。否则返回False")
    event: Optional[bool] = Field(default=None,description="Detailed Description of the Event")
    robot:Optional[str] = Field(default=None,description="机器人回复的消息")

def _get_datetime():
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")


from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个日程管家。请根据文本内容提取日程相关信息。"
            "如果你无法确定是需要提取的属性值，就返回null。",
        ),
        ("human", "{text}"),
        ("system", f"现在时间是：{_get_datetime()}"),
        ("robot","outmessage")
    ]
)

# 使用 ChatOpenAI 来生成回复
chat_openai = llm

def handle_input(text: str):
    # 使用 ChatOpenAI 生成回复
    response = chat_openai.generatePrompt(prompt.format(text=text))
    return Event(robot=response)



In [49]:
# 示例输入
input_text = "美女，你真美"

# 处理输入并获取回复
response = handle_input(input_text)

AttributeError: 'ChatOpenAI' object has no attribute 'generatePrompt'

“明天我可能会遇到意想不到的惊喜。”
“明天将会带来新的机遇和挑战。”
“明天是一个全新的日子，充满了希望和期待。”
“明天我将会迈向新的旅程，探索未知的世界。”
“明天将会带来意想不到的转折，改变一切。”
“明天我将会与陌生人展开有趣的对话。”
“明天将会是一个充满惊喜和发现的冒险。”
“明天我将会体验到前所未有的新鲜事物。”
“明天将会是一个充满希望和激情的日子。”
“明天我将会与朋友们共度难忘的时光。”

In [46]:
input_text = """明天是一个未知的新开始，充满了无限可能。"""
runnable.invoke({"text": input_text})

Event(time='2024-07-02 08:23:05', schedule=None, event=None, robot=None)

In [43]:
input_text = """明天我会去图书馆还书。"""
runnable.invoke({"text": input_text})

Event(time='2024-07-03', schedule=True, event_description='还书', robot='你的问题，我没有了解明白')

In [44]:
input_text = """明天将会带来意想不到的转折，改变一切"""
runnable.invoke({"text": input_text})

Event(time='2024-07-02 08:13:26', schedule=None, event_description=None, robot='你的问题，我没有了解明白')

In [39]:
input_text = """明天我要和朋友一起去公园散步。"""
runnable.invoke({"text": input_text})

Event(time='2024-07-03', schedule=True, event_description='和朋友一起去公园散步')

In [40]:
input_text = """明天下午三点，我有一个重要的会议"""
runnable.invoke({"text": input_text})

Event(time='2024-07-03 15:00:00', schedule=True, event_description='我有一个重要的会议')